In [ ]:
import os, time, random, logging , datetime , cv2 , csv , subprocess , json , math , glob , sys
import numpy as np

from utils import globo , xdv , tfh5 , sinet , yammet

def enhance_probabilities2(probs, threshold , power=2):
    #return np.where(probs < threshold, (probs ** power), 1 - (1 - probs) ** power) ## < th closer 0 | > th closer 1
    return np.where(probs < threshold, probs, 1 - (1 - probs) ** power) ## > th closer 1

### wavtry with chuncked xdv_test_bg_train_a 

In [ ]:
data2 = xdv.load_train_valdt_test_from_xdvtest_bg_train_a('train' , globo.CFG_WAV_TRAIN["sinet_fi_iter"])
data_vpath2 = [ data2[idx]['vpath'] for idx in range(len(data2))]
data_pesarr2 =  [ data2[idx]['p_es_array'] for idx in range(len(data2))]
data_sf2 =  [ data2[idx]['sf'] for idx in range(len(data2))]
data_ef2 =  [ data2[idx]['ef'] for idx in range(len(data2))]
data_label2 = [ data2[idx]['label'] for idx in range(len(data2))]

classes_list2 = xdv.get_index_per_label_from_filelist(data_vpath2)



In [ ]:
label = 'guns'
class_letter = 'B2'

#label = 'explosion'
#class_letter = 'G'   
print("XDV CLASS",label,class_letter)


def watch_this_interval2(idx, aas):
    vpath = data_vpath2[idx]
    sf = data_sf2[idx]
    ef = data_ef2[idx]
    label = data_label2[idx]
    print("\tWATCH THIS",os.path.basename(vpath),sf,ef,label)
    
    ## watch stuff
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5;thickness = 1;lineType = cv2.LINE_AA
    green = (0, 255, 0);red = (0, 0, 255)
    cap = cv2.VideoCapture(vpath)
    if not cap.isOpened():
        print("Error opening video file")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:break

        frame_count += 1
        if sf <= frame_count <= ef:
            cv2.putText(frame, str(label), (10, 30), font, fontScale, green, thickness, lineType)
            cv2.putText(frame, str(aas), (10, 60), font, fontScale, red, thickness, lineType)
            #cv2.imshow('Video', frame)
            #if cv2.waitKey(25) & 0xFF == ord('q'):break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## ANOMALY | 1

sin_tp , sin_fn = 0 , 0
yam_tp , yam_fn = 0 , 0
glob_tp , glob_fn = 0 , 0
for i,idx in enumerate(classes_list2[class_letter]):
    
    vpath = data_vpath2[idx]
    sf = data_sf2[idx]
    ef = data_ef2[idx]
    
    p_es_arr = data_pesarr2[idx]
    p_es_arr_max = np.max(p_es_arr,axis=0)
    
    p_es_arr_enh = enhance_probabilities2(p_es_arr , 0.4)
    p_es_arr_enh_max = np.max(p_es_arr_enh,axis=0) 
    
    p_yam_arr = yammet.get_sigmoid(vpath,sf,ef)
    p_yam_arr_max = np.max(p_yam_arr,axis=0)
    #sorted_idx = np.argsort(p_yam_arr_max)[::-1]
    #for k in range(50):print('- {}: {:.4f}'.format(np.array(globo.CFG_YAMMET['labels'])[sorted_idx[k]], predictions[sorted_idx[k]]))
    
    
    print("\n",os.path.basename(vpath))
    print("YAMMET")
    p_yam_arr_anom = []
    yam_tp_flag , yam_fn_flag = False , False
    for kkk,anom_yam_k in enumerate(globo.CFG_YAMMET["anom_labels_i"]):
        if p_yam_arr_max[anom_yam_k] >= 0.4:
            print('-TP- {}: {:.4f}'.format(np.array(globo.CFG_YAMMET['labels'])[anom_yam_k], p_yam_arr_max[anom_yam_k]))
            yam_tp_flag = True
            yam_fn_flag = False
            break
        else:
            print('-FN- {}: {:.4f}'.format(np.array(globo.CFG_YAMMET['labels'])[anom_yam_k], p_yam_arr_max[anom_yam_k]))
            yam_fn_flag = True
            
        p_yam_arr_anom.append(p_yam_arr_max[anom_yam_k])
    
    if yam_tp_flag: yam_tp += 1
    if yam_fn_flag: yam_fn += 1
    
    print("SINET")
    p_sin_arr_anom = []
    sin_tp_flag , sin_fn_flag = False , False
    for kkk,anom_sin_k in enumerate(globo.CFG_SINET["anom_labels_i2"]):
        if p_es_arr_enh_max[anom_sin_k] >= 0.5:
            print('-TP- {}: {:.4f}  ,  {:.4f}'.format(np.array(globo.CFG_SINET['labels'])[anom_sin_k], p_es_arr_max[anom_sin_k] , p_es_arr_enh_max[anom_sin_k]))
            sin_tp_flag = True
            sin_fn_flag = False
            break
        else :
            print('-FN- {}: {:.4f}  ,  {:.4f}'.format(np.array(globo.CFG_SINET['labels'])[anom_sin_k], p_es_arr_max[anom_sin_k] , p_es_arr_enh_max[anom_sin_k]))
            sin_fn_flag = True
            
        p_sin_arr_anom.append(p_es_arr_enh_max[anom_sin_k])
    
    if sin_tp_flag: sin_tp += 1
    if sin_fn_flag: sin_fn += 1
    
    ## global
    if yam_tp_flag or sin_tp_flag: glob_tp += 1
    if yam_fn_flag and sin_fn_flag: glob_fn +=1
    
print("SINNET\n","tp",sin_tp,"fn",sin_fn)
print("YAMMET\n","tp",yam_tp,"fn",yam_fn)
print("GLOBAL\n","tp",glob_tp,"fn",glob_fn)
print(len(classes_list2[class_letter]))

In [ ]:
## NORMAL

tn0 , fp0 = 0 , 0
tn1 , fp1 = 0 , 0

for i,idx in enumerate(classes_list2['A']):
    #print("\n",os.path.basename(data_vpath2[idx]))
    
    p_es_arr = data_pesarr2[idx]
    #p_es_arr_enh = enhance_probabilities2(p_es_arr , 0.5)
    
    max = np.max(p_es_arr,axis=0)[iii]
    #max_enh = np.max(p_es_arr_enh,axis=0)[iii]
    
    #print(os.path.basename(data_vpath2[idx]))
    
    if max >= 0.5: 
        fp0 += 1
        print("FP")
        #for j in range(len(p_es_arr[:,iii])):
        #    print(round(p_es_arr[j,iii],3))
        watch_this_interval2(idx,round(max,3))
    else: tn0 += 1
       
    #if max_enh >= 0.5: fp1 += 1
    #else: tn1 += 1
 
print("ORIGINAL\n","tn",tn0,"fp",fp0)
#print("ENHANCED\n","tn",tn1,"fp",fp1)

### TEST BG WITHOUT CHUNCK

In [ ]:
mode = 'train'

data1 = np.load(os.path.join(globo.AAS_PATH+"/1_full_interval",f"{globo.CFG_SINET['sinet_version']}--fl1_{mode}.npz"), allow_pickle=True)["data"]
data_vpath1 = [ data1[idx]['vpath'] for idx in range(len(data1))]
data_pesarr1 =  [ data1[idx]['p_es_array'] for idx in range(len(data1))]
data_sf1 =  [ data1[idx]['frame_interval'][0] for idx in range(len(data1))]
data_ef1 =  [ data1[idx]['frame_interval'][1] for idx in range(len(data1))]
data_label1 = [ data1[idx]['frame_interval'][2] for idx in range(len(data1))]

classes_list1 = xdv.get_index_per_label_from_filelist(data_vpath1)

In [ ]:

def watch_this_interval1(idx, aas):
    vpath = data_vpath1[idx]
    sf = data_sf1[idx]
    ef = data_ef1[idx]
    label = data_label1[idx]
    print("\tWATCH THIS",os.path.basename(vpath),sf,ef,label)
    
    ## watch stuff
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5;thickness = 1;lineType = cv2.LINE_AA
    green = (0, 255, 0);red = (0, 0, 255)
    cap = cv2.VideoCapture(vpath)
    if not cap.isOpened():
        print("Error opening video file")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:break

        frame_count += 1
        if sf <= frame_count <= ef:
            cv2.putText(frame, str(label), (10, 30), font, fontScale, green, thickness, lineType)
            cv2.putText(frame, str(aas), (10, 60), font, fontScale, red, thickness, lineType)
            #cv2.imshow('Video', frame)
            #if cv2.waitKey(1) & 0xFF == ord('q'):break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## ANOMALY | 1

tp0 , fn0 = 0 , 0
tp1 , fn1 = 0 , 0

for i,idx in enumerate(classes_list1[class_letter]):
    if data_label1[idx] == 1:
        #print("\n",os.path.basename(data_vpath2[idx]))    
        vpath = data_vpath1[idx]
        
        p_es_arr = data_pesarr1[idx]
        p_es_arr_enh = enhance_probabilities(p_es_arr , 0.4)
        
        ## AVERAGE OF THE TOPK PROBS 
        p_es_arr_iii = p_es_arr[:,iii]
        p_es_arr_iii_sorted = np.argsort(p_es_arr_iii)[::-1]
        topk = int(len(p_es_arr_iii)/2)
        p_es_arr_iii_sorted_topk=p_es_arr_iii_sorted[:topk]
        avg_topk = np.mean(p_es_arr[p_es_arr_iii_sorted_topk, iii])

        ## AVERAGE
        avg = np.average(p_es_arr,axis=0)[iii]
        
        ## MAX
        max = np.max(p_es_arr,axis=0)[iii]
        max_enh = np.max(p_es_arr_enh,axis=0)[iii]
        
        #print(label,round(max,3),"|",round(max_enh,3))
        
        if max >= 0.5: tp0 += 1
        else: fn0 += 1
        
        if max_enh >= 0.5: 
            tp1 += 1
            #print("TP",label,round(max,3),"|",round(max_enh,3))
        else: 
            print("FN",label,\
                  "\n\tMAX",round(max,3),"|",round(max_enh,3) ,\
                    "\n\tAVG",round(avg,3) , round(avg_topk,3))
            #watch_this_interval1(idx,max_enh)
            
            fn1 += 1
 
print("ORIGINAL\n","tp",tp0,"fn",fn0)
print("ENHANCED\n","tp",tp1,"fn",fn1)

In [ ]:
## NORMAL

tn0 , fp0 = 0 , 0
tn1 , fp1 = 0 , 0

for i,idx in enumerate(classes_list1['A']):
    
        print("\n",os.path.basename(data_vpath2[idx]))
        
        p_es_arr = data_pesarr1[idx]
        p_es_arr_enh = enhance_probabilities(p_es_arr , 0.5)
        
        maxx = np.max(p_es_arr,axis=0)[iii]
        max_enh = np.max(p_es_arr_enh,axis=0)[iii]
        
        #print(label,round(max,3),"|",round(max_enh,3))
        
        if maxx >= 0.5: fp0 += 1
        else: tn0 += 1
        
        if max_enh >= 0.5: 
            fp1 += 1
            print("FP",label,round(maxx,3),"|",round(max_enh,3))
        else: tn1 += 1
 
print("ORIGINAL\n","tn",tn0,"fp",fp0)
print("ENHANCED\n","tn",tn1,"fp",fp1)

### XDV TRAIN DATASET ORIGINAL , AUDIO RUN TRY